## Parquet to SensorThings Streamlike

In [2]:
import csv
import re
import math
import time
import random
import numpy as np
import sys
import json
import requests
import os
import Helper

In [6]:
# dirList: sorted list of dir names
dirList = os.listdir("../dataToSensorThing/outDir/outAllSortByTimeStampAndIDBigAll")
dirList= [ x for x in dirList if "201601" in x ]
dirList = sorted(dirList, key= lambda myDir: int(myDir.split("=")[1]))



In [7]:
dirList

['TimeStamp=20160101',
 'TimeStamp=20160102',
 'TimeStamp=20160103',
 'TimeStamp=20160104',
 'TimeStamp=20160105',
 'TimeStamp=20160106',
 'TimeStamp=20160107',
 'TimeStamp=20160108',
 'TimeStamp=20160109',
 'TimeStamp=20160110',
 'TimeStamp=20160111',
 'TimeStamp=20160112',
 'TimeStamp=20160113',
 'TimeStamp=20160114',
 'TimeStamp=20160115',
 'TimeStamp=20160116',
 'TimeStamp=20160117',
 'TimeStamp=20160118',
 'TimeStamp=20160119',
 'TimeStamp=20160120',
 'TimeStamp=20160121',
 'TimeStamp=20160122',
 'TimeStamp=20160123',
 'TimeStamp=20160124',
 'TimeStamp=20160125',
 'TimeStamp=20160126',
 'TimeStamp=20160127',
 'TimeStamp=20160128',
 'TimeStamp=20160129',
 'TimeStamp=20160130',
 'TimeStamp=20160131']

In [8]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
#spark conf
conf = ( SparkConf()
         .setMaster("local[6]")
         .setAppName('pyspark')
        )
ss = SparkSession.builder.config(conf=conf).getOrCreate()
sc = ss.sparkContext


In [4]:
#Common things
mytypes = ["Air-Temperature","Air-Humidity","Air-Pressure"]
urlHome = 'http://swarm-node01.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
urlSensors = urlHome + '/Sensors'
urlObservedProperty = urlHome + '/ObservedProperties'
urlDataStream = urlHome + '/Datastreams'
urlFoI = urlHome + '/FeaturesOfInterest'

In [5]:
myPrefix = ""

from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime

myDir = dirList[0]
inputDir = "../dataToSensorThing/outDir/outAllSortByTimeStampAndIDBigAll/"+myDir
dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
idDF = dataFileDF["ID","Latitude","Longitude"]

urlInst = Helper.Helper.Inst_URL
idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

idDF = idDF.withColumn("ThUID",F.format_string("Things/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
idDF = idDF.withColumn("SensorUID",F.format_string("Sensors/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
idDF = idDF.withColumn("DsUID",F.format_string("Datastreams/%s/MS/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))


idDF.rdd.first()

## Register ObservedProperties

In [13]:
#Observed Property
for mytype in mytypes:
    data ={
        "name": "Air-Temperature",
        "description": "The degree or intensity of heat present in the area",
        "definition": "http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature",
        "@iot.id": myPrefix + "ObservedProperty/" + mytype
    }
    p = requests.post(urlObservedProperty, json.dumps(data))
    print(data)
    if (p.status_code  == 201):
        print(201)
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': '1ObservedProperty/Air-Temperature', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201
{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': '1ObservedProperty/Air-Humidity', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201
{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': '1ObservedProperty/Air-Pressure', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201


In [7]:
from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime

def registerEntities(row):
    #Thing
    
    data =  {
        "name": "WG-WeatherStation-" + row.ID,
        "description": "This is DWD-Sensor-" + row.ID,
        "@iot.id": myPrefix + row.ThUID,
        "Locations": [
        {
            "name": row.ThUID,
            "description": "This is the location of WG-Sensor-" + row.ID,
            "encodingType": "application/vnd.geo+json",
            "location": {
              "type": "Point",
              "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
            },
            "@iot.id": myPrefix+"Location/" + row.ThUID 
        }
      ]
        }
    p = requests.post(urlThings, json.dumps(data))
    print("Thing:")
    print(data)
    if (p.status_code  == 201):
        print(201)
        
    else:
        print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #Sensor

    deviceAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row.ID
    for mytype in mytypes:
        data = {
            "name": "WG-WeatherStation-" + row.ID + "-" + "" + mytype,
            "description": "This is a Sensor from Netatmo Weather Station",
            "encodingType": "application/pdf",
            "metadata": deviceAddr,
            "@iot.id": myPrefix+row.SensorUID +"-"+ mytype
        }
        p = requests.post(urlSensors, json.dumps(data))
        print("Sensor:")
        print(data)
        if (p.status_code  == 201):
            print(201)
            
            
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #DataStream

    for mytype in mytypes:
        data = {
          "name": "MS/LiveMeasure",
          "description": "Datastream for recording temperature",
          "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
          "@iot.id":myPrefix + row.DsUID + "-" + mytype + "/MS/LiveMeasure",
          "unitOfMeasurement": {
            "name": "Degree Celsius",
            "symbol": "degC",
            "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius"
          },
          "Thing":{"@iot.id":myPrefix+row.ThUID},
          "ObservedProperty":{"@iot.id":myPrefix+"ObservedProperty/" + mytype},
          "Sensor":{"@iot.id":myPrefix+row.SensorUID +"-"+ mytype}
        }
        p = requests.post(urlDataStream, json.dumps(data))
        print("DS:")
        print(data)
        if (p.status_code  == 201):
            print(201)
            
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    # FoI
    data = {
      "name": "Weather Station-" + row.ID,
      "description": "Weather Station-" + row.ID,
      "encodingType": "application/vnd.geo+json",
        "@iot.id": myPrefix +"FoI/" +myPrefix+ row.DsUID + "-Air-Temperature/MS/LiveMeasure/"+row.LatLon[0]+":"+row.LatLon[1],
      "feature": {
        "type": "Point",
        "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
      }
    }
    p = requests.post(urlFoI, json.dumps(data))
    print("FOI:")
    print(data)
    if (p.status_code  == 201):
        print(201)
        
        
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)




for myDir in dirList:
    inputDir = "../dataToSensorThing/outDir/outAllSortByTimeStampAndIDBigAll/"+myDir
    dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
    dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
    dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
    idDF = dataFileDF["ID","Latitude","Longitude"]
    
    urlInst = Helper.Helper.Inst_URL
    idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
    idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

    idDF = idDF.withColumn("ThUID",F.format_string("Thing/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("SensorUID",F.format_string("Sensor/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("DsUID",F.format_string("Datastream/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))


    for hourInt in range(0,24):
        oneHourDataFileDF = dataFileDF.rdd.filter(lambda row: int(row.Time.split(':')[0]) == hourInt)
        print(oneHourDataFileDF.count())
        for row in oneHourDataFileDF.collect():
            

            class TZ(tzinfo):
                def utcoffset(self, dt): return timedelta(minutes=120)
            isoTime =datetime(int(row.Date.split('-')[0]),
                         int(row.Date.split("-")[1]),
                         int(row.Date.split("-")[2]),
                         int(row.Time.split(':')[0]),
                         int(row.Time.split(':')[1]),
                         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()




            urlObs = urlHome+"/Datastreams('"+myPrefix+"Datastream/" + urlInst + "/WG-WeatherStation-" +row.ID+ "-" + "Air-Temperature/MS/LiveMeasure')/Observations"
            print(urlObs)
            data = {
              "phenomenonTime": isoTime,
              "resultTime" : isoTime,
              "result" : row.AirTemperature,
              "FeatureOfInterest": {
                  "@iot.id": myPrefix +"FoI/" + myPrefix + "Datastream/"+urlInst +"/WG-WeatherStation-"+row.ID + "-Air-Temperature/MS/LiveMeasure/" +row.Latitude+":"+row.Longitude
              }
            }
            p = requests.post(urlObs, json.dumps(data))

            if (p.status_code  == 201):
                print("+1")
                print(p.headers["location"])
            else:
                print("Error:", p.status_code)
                #print(data)
                #print(idDF.rdd.take(5))
                registerEntities(idDF.rdd.filter(lambda aRow:aRow.ID == row.ID).first())

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)
    time.sleep(10)
    

14761
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature/MS/LiveMeasure')/Observations
('Error:', 400)
Thing:
{'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-ILDINGHA4', 'Locations': [{'location': {'type': 'Point', 'coordinates': [51.776, 7.454]}, '@iot.id': u'1Location/Thing/edu.teco.wang/WG-WeatherStation-ILDINGHA4', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-ILDINGHA4', 'description': u'This is the location of WG-Sensor-ILDINGHA4'}], 'name': u'WG-WeatherStation-ILDINGHA4', 'description': u'This is DWD-Sensor-ILDINGHA4'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=ILDINGHA4', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature', 'name': u'WG-WeatherStation-ILDINGHA4-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather S

+1
http://localhost:8080/FROST-Server/v1.0/Observations('3e825eb6-7a1b-11e8-acf2-8776e57362d2')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-ITAMBACH2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('3e865480-7a1b-11e8-acf2-13d9ba95331b')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-ITAMBACH2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('3e89b922-7a1b-11e8-acf2-c757cf84da06')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-ITAMBACH2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('3e8ccbee-7a1b-11e8-acf2-6b2e2a719e33')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IREMSHAL7-Ai

FOI:
{'feature': {'type': 'Point', 'coordinates': [49.573, 12.418]}, '@iot.id': u'1FoI/1Datastream/edu.teco.wang/WG-WeatherStation-IMOOSBAC2-Air-Temperature/MS/LiveMeasure/49.573:12.418', 'encodingType': 'application/vnd.geo+json', 'name': u'Weather Station-IMOOSBAC2', 'description': u'Weather Station-IMOOSBAC2'}
201
No such entity 'Datastream' with id 1Datastream/edu.teco.wang/WG-WeatherStation-IMOOSBAC2-Air-Temperature/MS/LiveMeasure
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IMOOSBAC2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('41fa1f8e-7a1b-11e8-acf2-07a047eaafc5')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IMOOSBAC2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('42024fba-7a1b-11e8-acf2-a7aba620c6eb')
http://swarm-node01.teco.edu:8080/FROS

FOI:
{'feature': {'type': 'Point', 'coordinates': [51.285, 7.25]}, '@iot.id': u'1FoI/1Datastream/edu.teco.wang/WG-WeatherStation-IWUPPERT89-Air-Temperature/MS/LiveMeasure/51.285:7.250', 'encodingType': 'application/vnd.geo+json', 'name': u'Weather Station-IWUPPERT89', 'description': u'Weather Station-IWUPPERT89'}
201
No such entity 'Datastream' with id 1Datastream/edu.teco.wang/WG-WeatherStation-IWUPPERT89-Air-Temperature/MS/LiveMeasure
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IWUPPERT89-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('43e484ec-7a1b-11e8-acf2-f7bb476da71a')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IWUPPERT89-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('43e79704-7a1b-11e8-acf2-9362b26fd519')
http://swarm-node01.teco.edu:8080/F

DS:
{'observationType': 'http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement', '@iot.id': u'1Datastream/edu.teco.wang/WG-WeatherStation-IWAIBLIN46-Air-Pressure/MS/LiveMeasure', 'name': 'MS/LiveMeasure', 'unitOfMeasurement': {'definition': 'http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius', 'symbol': 'degC', 'name': 'Degree Celsius'}, 'Thing': {'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-IWAIBLIN46'}, 'ObservedProperty': {'@iot.id': '1ObservedProperty/Air-Pressure'}, 'Sensor': {'@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-IWAIBLIN46-Air-Pressure'}, 'description': 'Datastream for recording temperature'}
201
FOI:
{'feature': {'type': 'Point', 'coordinates': [48.819, 9.311]}, '@iot.id': u'1FoI/1Datastream/edu.teco.wang/WG-WeatherStation-IWAIBLIN46-Air-Temperature/MS/LiveMeasure/48.819:9.311', 'encodingType': 'application/vnd.geo+json', 'name': u'Weather Station-IWAIBLIN46', 'description': u'Weather Station-IWAIBLIN46'}
201
No such entity

...

Thing:
{'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-IALTENST3', 'Locations': [{'location': {'type': 'Point', 'coordinates': [50.289, 8.961]}, '@iot.id': u'1Location/Thing/edu.teco.wang/WG-WeatherStation-IALTENST3', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IALTENST3', 'description': u'This is the location of WG-Sensor-IALTENST3'}], 'name': u'WG-WeatherStation-IALTENST3', 'description': u'This is DWD-Sensor-IALTENST3'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IALTENST3', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-IALTENST3-Air-Temperature', 'name': u'WG-WeatherStation-IALTENST3-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IALTENST3', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG

Thing:
{'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-IOLCHING18', 'Locations': [{'location': {'type': 'Point', 'coordinates': [48.224, 11.348]}, '@iot.id': u'1Location/Thing/edu.teco.wang/WG-WeatherStation-IOLCHING18', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IOLCHING18', 'description': u'This is the location of WG-Sensor-IOLCHING18'}], 'name': u'WG-WeatherStation-IOLCHING18', 'description': u'This is DWD-Sensor-IOLCHING18'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IOLCHING18', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-IOLCHING18-Air-Temperature', 'name': u'WG-WeatherStation-IOLCHING18-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IOLCHING18', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.t

Thing:
{'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-IGANDERK5', 'Locations': [{'location': {'type': 'Point', 'coordinates': [53.04, 8.564]}, '@iot.id': u'1Location/Thing/edu.teco.wang/WG-WeatherStation-IGANDERK5', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IGANDERK5', 'description': u'This is the location of WG-Sensor-IGANDERK5'}], 'name': u'WG-WeatherStation-IGANDERK5', 'description': u'This is DWD-Sensor-IGANDERK5'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IGANDERK5', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-IGANDERK5-Air-Temperature', 'name': u'WG-WeatherStation-IGANDERK5-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IGANDERK5', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-

+1
http://localhost:8080/FROST-Server/v1.0/Observations('add91908-7a1b-11e8-acf2-7f8706de65c6')
http://swarm-node01.teco.edu:8080/FROST-Server/v1.0/Datastreams('1Datastream/edu.teco.wang/WG-WeatherStation-IBIEBESH2-Air-Temperature/MS/LiveMeasure')/Observations
('Error:', 400)
Thing:
{'@iot.id': u'1Thing/edu.teco.wang/WG-WeatherStation-IBIEBESH2', 'Locations': [{'location': {'type': 'Point', 'coordinates': [49.779, 8.464]}, '@iot.id': u'1Location/Thing/edu.teco.wang/WG-WeatherStation-IBIEBESH2', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IBIEBESH2', 'description': u'This is the location of WG-Sensor-IBIEBESH2'}], 'name': u'WG-WeatherStation-IBIEBESH2', 'description': u'This is DWD-Sensor-IBIEBESH2'}
201
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IBIEBESH2', 'encodingType': 'application/pdf', '@iot.id': u'1Sensor/edu.teco.wang/WG-WeatherStation-IBIEBESH2-Air-Temperature', 'name': u'WG-WeatherS

Exception: could not open socket

In [ ]:
Datastream/edu.teco.wang/WG-WeatherStation-IBAESWEI4-Air-Temperature/MS/LiveMeasure
Datastream/edu.teco.wang/WG-WeatherStation-IBAESWEI4-Air-Temperature/MS/LiveMeasure

/FoI/Datastream/edu.teco.wang/WG-WeatherStation-IBIRKENH15-Air-Temperature/MS/LiveMeasure/49.480:8.265

In [ ]:
urlObs = urlHome+"/Datastreams('"+myPrefix+"edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"

p = requests.post(urlObs, json.dumps(
{
  "phenomenonTime": isoTime,
  "resultTime" : isoTime,
  "result" : row.AirTemperature,
  "FeatureOfInterest": {
      "@iot.id": myPrefix + urlInst+"/"+row.ID + "/FoI/"+row.ID+"/"+row.Latitude+":"+row.Longitude
  }
}
))

if (p.status_code  == 201):
    print(p.headers["location"])
else:
    print("Error:", p.status_code)

for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:

#idDF = idDF.withColumn("FoIUID",F.format_string("%s/FeaturesOfInterest/%s",idDF.ThID,idDF.ID))
#idDF = idDF.withColumn("SensorUID",F.format_string("%s/Sensors/%s",idDF.ThID,idDF.ID))


In [ ]:
row = idDF.collect()[0]

row

In [ ]:
# Observations
dataFileDF.show(10)

In [ ]:
row = dataFileDF.collect()[0]
row

In [ ]:
from datetime import tzinfo, timedelta, datetime
for row in dataFileDF.collect():
    

    class TZ(tzinfo):
        def utcoffset(self, dt): return timedelta(minutes=120)
    isoTime =datetime(int(row.Date.split('-')[0]),
                 int(row.Date.split("-")[1]),
                 int(row.Date.split("-")[2]),
                 int(row.Time.split(':')[0]),
                 int(row.Time.split(':')[1]),
                 int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()
    
    
    
    
    urlObs = urlHome+"/Datastreams('edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
    data ={
      "phenomenonTime": isoTime,
      "resultTime" : isoTime,
      "result" : row.AirTemperature
    }
    p = requests.post(urlObs, json.dumps(data))
    if (p.status_code  == 201):
        print(p.headers["location"])
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)


In [ ]:
urlObs

In [ ]:
p.headers["location"]

In [ ]:
from datetime import tzinfo, timedelta, datetime
row = dataFileDF.first()
row.Date.split("-")[0]
class TZ(tzinfo):
    def utcoffset(self, dt): return timedelta(minutes=120)
datetime(int(row.Date.split('-')[0]),
         int(row.Date.split("-")[1]),
         int(row.Date.split("-")[2]),
         int(row.Time.split(':')[0]),
         int(row.Time.split(':')[1]),
         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()

## Tryouts

In [ ]:
row = idDF.collect()[1]
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
data =  {
        "name": "DWD-Sensor-noSens" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/noSensblblb" + row[0]
    }
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print(201)
else:
    print("Error:", p.status_code)
    
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:

idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))#.show(3)#.agg(F.first(idDF.Longitude))

In [ ]:
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'

In [ ]:

#idDF2 = idDF.select("ID","LatLon").rdd.map(DfToSensorthings)

In [ ]:
row = idDF.collect()[1]
import requests
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
sensorAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row[0]
data =  {
        "name": "DWD-Sensor-" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/" + row[0]
    }
p = requests.get(urlThings+"('DWD-Sensor/noSensblbl" + row[0]+"')")
if (p.status_code  == 200):
    print(200)
else:
    print("Error:", p.status_code)
    
for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:
row

In [ ]:
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print("Creation successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)
        

In [ ]:
#Delete a thing

deleteID = urlThings + "('DWD-Sensors/IBREUNA2')"
print(deleteID)
p = requests.delete(urlThings)
if (p.status_code  == 201):
    print("Deletion successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
#Query a thing

getID = urlThings + "(\"teco.edu/Test-2\")"
print(getID)
p = requests.get(urlThings)
if (p.status_code  == 201):
    print("Get successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
p.content

In [ ]:
sensors = idDF.collect()
for sensor in sensors:
    print(DfToSensorthingsPost(sensor))
    #self link

In [ ]:
import requests

In [ ]:
dataFileDF = ss.read.parquet(inputDir)
outDir = "./outDir/outAllAll5/"
print(csv.__file__)

In [ ]:
validFileNames = [inputDir+f for f in os.listdir(inputDir) if ('.' not in f) and ("part-" in f)]
validFileNames

In [ ]:
for aFile in validFileNames:
    with open(aFile) as csvfile:
        for row in csvfile:
            k = row[1:9]
            v = row[13:-3]+'\n'
            vNew = re.sub(';','\n',v)
            outFile = open(outDir+k,"w")
            outFile.write(vNew)
            outFile.close()


In [ ]:
row = csvfile.readline()

In [ ]:
k = row[1:9]
v = row[13:-3].split(";")
k,v

In [ ]:
with open(inputFile) as csvfile:
    for row in csvfile:
        k = row[1:9]
        v = row[13:-3]+'\n'
        vNew = re.sub(';','\n',v)
        outFile = open(outDir+k,"w")
        outFile.write(vNew)
        outFile.close()

In [ ]:
import re

print(vNew)

In [ ]:
outFile = open(outDir+k,"w")
outFile.write(vNew)
outFile.close()

In [ ]:
with open(inputFile,'rb') as csvfile:
    reader = csv.Reader(csvfile)
    row = reader.__next__()
row

In [ ]:
row

In [ ]:
all=string.maketrans('','')
nodigs=all.translate(all, string.digits)

In [ ]:
class Del:
  def __init__(self, keep=string.digits):
    self.comp = dict((ord(c),c) for c in keep)
  def __getitem__(self, k):
    return self.comp.get(k)

In [ ]:
with open(inputPath) as csvfile:
    reader = csv.DictReader(csvfile)
    for i in range(10):
        row = reader.__next__()
        
        lineID = row['id']
        timeStamp = row['time'].translate(Del())
        print(lineID, timeStamp)
        